In [1]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.8.9\n")
    f.write("requests\n")
    
!pip install -r requirements.txt  --upgrade --user

     |████████████████████████████████| 296 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 12.7 MB 70.2 MB/s eta 0:00:01         | 5.9 MB 70.2 MB/s eta 0:00:01
  Created wheel for kfp: filename=kfp-1.8.9-py3-none-any.whl size=409653 sha256=797c9a17a6c1c920cb4916aaa74d5330370b39d5253e9d5c2aea99b42f665e7b
  Stored in directory: /home/jovyan/.cache/pip/wheels/74/45/05/323f17840bbf9963e873b7a1e75caba3d5ab927c390c98c740
Successfully built kfp
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [12]:
import kfp.dsl as dsl
import kfp
from kfp import components

kserve_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')

@dsl.pipeline(
  name='KFServing pipeline',
  description='A pipeline for KFServing.'
)
def kfservingPipeline(
    action='apply',
    model_name='tensorflow-sample',
    model_uri='gs://kfserving-examples/models/tensorflow/flowers',
    namespace='kubeflow-user-example-com',
    framework='tensorflow',
    runtime_version='v0.5.1-debug-1',
    ):
    
    # flowers model was trained with 5 category: [daisy(雛菊)  dandelion(浦公英)  roses(玫瑰)  sunflowers(向日葵)  tulips(鬱金香)]
    # see details here:  https://github.com/tensorflow/hub/blob/master/examples/colab/image_feature_vector.ipynb


    isvc_yaml = '''
apiVersion: "serving.kubeflow.org/v1beta1"
kind: "InferenceService"
metadata:
  name: {0}
  namespace: {1}
spec:
  predictor:
    tensorflow:
      storageUri: {2}
      image: tensorflow/serving:2.8.0-gpu
  explainer:
    aix:  
      runtimeVersion: {3}
      type: LimeImages
      config:
        num_samples: "100"
        top_labels: "10"
        min_weight: "0.01"
'''.format(model_name, namespace, model_uri, runtime_version)
    kserve_op(
        action=action,
        inferenceservice_yaml=isvc_yaml
    )

In [13]:
# Compile pipeline
kfp.compiler.Compiler().compile(kfservingPipeline, 'tf-flower.zip')

Please upload the manifests generated above(tf-flower.zip)
And Below is example scripts to send requests to the model you deployed
Or you can run this on your host machines under hack/hack/local-test-with-kfserving.py

##### this is fixed in v1.3.1, so you are no longer to take actions
~note: please disable sidecar-injection on kubeflow-user namespace,
as knative-serving / knative-events are not enabled istio sidecar.
failed to disable could get RBAC error~


##### you have to REPLACE authervice_session token into yours. Go to Browser -> Developer Console -> Application -> Cookies
![title](img/cookies.png)

In [ ]:
####
#### few hackings are required to run this aix with LIME and tensorflow flower classification
#### 1. use aix model from footprintai
####   before our patch are merged to kserve, you need to modify configmap to use our image for aix
####   >> kubectl -n kubeflow edit configmaps inferenceservice-config
####   and replace value with the name "explainers.aix.image" to be "footprintai/aixserver", and runtime version (specified during run the pipeline with v0.5.1-debug-25)
#### 2. create a run for this pipeline (which will provision inferenceservices with AIX) with parameters
####   model_name: tensorflow-sample
####   model_uri: gs://kfserving-examples/models/tensorflow/flowers
####   runtime_version: v0.5.1-debug-25
#### 3. then wait for kserve ready
####   >> -n kubeflow-user-example-com get pods
####   tensorflow-sample-explainer-default-00010-deployment-694799lmwl   3/3     Running     0          9m21s
####   tensorflow-sample-predictor-default-00001-deployment-5dff6hmzzs   3/3     Running     0          2d8h
#### 4. then send requests to explain url


In [45]:
import sys
import base64
import requests
import json
import math
from matplotlib import pyplot as plt
import numpy as np
import time
from skimage.color import gray2rgb, label2rgb 

print('************************************************************')
print('************************************************************')
print('************************************************************')
print("starting query")


endpoint = "http://istio-ingressgateway.istio-system/v1/models/tensorflow-sample:explain"
headers = {
    'Host': "tensorflow-sample.kubeflow-user-example-com.example.com",
    "Cookie": "authservice_session=MTY1NjgyOTE1M3xOd3dBTkZaVVRVZFVOMUpDV1VkUFRFOUlUbEJSUkRWR1VsWlNUMVJJVjFKQlJVcFBWMDFOTkZBMVFqZE9Oa3BNUkVKWFdreFZVRkU9fBb0U-LBvUREcLO5meF_x6ci93GAhGw8aO8G6bjLtYs7"
}

with open('floribunda.jpg', 'rb') as f:
    image_content = f.read()
    image_64_encode = base64.encodebytes(image_content).decode('utf-8')
    
payload = {"instances":[{"image_bytes": {"b64": image_64_encode}, "key": "1"}], "top_labels": "2", "min_weight": "0.001"}


print("Sending Explain Query")

x = time.time()

res = requests.post(endpoint, json=payload, headers=headers)

print("TIME TAKEN: ", time.time() - x)

print(res)
if not res.ok:
    res.raise_for_status()


************************************************************
************************************************************
************************************************************
starting query
Sending Explain Query
TIME TAKEN:  35.64965891838074
<Response [200]>


In [ ]:
actual="2"


# flowers model was trained with 5 category: [daisy(雛菊)  dandelion(浦公英)  roses(玫瑰)  sunflowers(向日葵)  tulips(鬱金香)]
# see details here:  https://github.com/tensorflow/hub/blob/master/examples/colab/image_feature_vector.ipynb

    
res_json = res.json()
temp = np.array(res_json["explanations"]["temp"])
masks = np.array(res_json["explanations"]["masks"])
top_labels = np.array(res_json["explanations"]["top_labels"])


from PIL import Image
 
# Read image
img = np.array(Image.open('floribunda.jpg'))



print("top_labels:%s",top_labels)
fig, m_axs = plt.subplots(2, 2, figsize=(12, 6))
for i, c_ax in enumerate(m_axs.flatten()):
     
    if i >= len(top_labels):
        c_ax.axis('off')
        continue
    mask = masks[i]
    c_ax.imshow(label2rgb(mask, img, bg_label=0), interpolation='nearest')
    c_ax.set_title('Positive for {}\nActual {}'.format(top_labels[i], actual))
    c_ax.axis('off')
plt.show()